# Projeto 2 - Ciência dos Dados

Nome: Pedro Henrique Menezes de Oliveira

Nome: Lucio Hallage

Nome: Vitor Miada

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [226]:
import tweepy
import math
import os.path
import pandas as pd
import json
import numpy as np
from random import shuffle

___
## Autenticando no  Twitter

* Conta: @pedrohmenezeso

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


Capturando os dados do twitter:

Salvando os dados em uma planilha Excel:

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

**Irrelevante - 0:** Tweets que não estão relacionados diretamente com a Netflix. Ex.: "*malta digam ai séries para ver na netflix*"

**Relevante - 1:** Tweets que estão relacionados diretamente com a Netflix e que dizem algo a respeito que expresse opinião. Ex.: "*50 tons mais escuros vai ta na netflix nen adoro😅*"

**Neutros - 2:** Tweets que estão relacionados diretamente com a Netflix porém não expressam opinião. Ex.: "*milhões de séries disponíveis na netflix,mas minha preferência é sempre #glee!!!*"

In [227]:
base=pd.ExcelFile('NetflixNrt.xlsx')
data=pd.read_excel(base,'Treinamento')
data_indexado=data.set_index('Treinamento')
data_indexado

,Relevância,Unnamed: 2,Unnamed: 3
Treinamento,,,
"milhões de séries disponíveis na netflix,mas minha preferência é sempre #glee!!!",2,NaN,NaN
@buriedslive vai se fuder e me empresta sua netflix,0,NaN,NaN
eu assisti 5 episódios de uma série na netflix e percebi que tava no 4g vsf,0,NaN,NaN
"estou assistindo as telefonistas, stranger things e how to get away with murder, além de ter vis a vis no backlog. daí vem a netflix e me lança nova temporada de elite! não to preparada!",1,NaN,NaN
50 tons mais escuros vai ta na netflix nen adoro😅,1,NaN,NaN
to tão por fora de tudo que eu vi hoje que saiu greys na netflix,2,NaN,NaN
malta digam ai séries para ver na netflix,0,NaN,NaN
@jcarloscr todas as séries originais amazon que eu assisti tinham uma qualidade superior na parte técnica. e gosto bastante dos roteiros mais diferenciados. é bem menos óbvio que a netflix,0,NaN,NaN
e minha netflix q aparece que lancou a 15 temporada de greys e a 2 de elite e nada delas la,2,NaN,NaN


In [228]:
data_indexado.index

Index(['milhões de séries disponíveis na netflix,mas minha preferência é sempre #glee!!!',
       '@buriedslive vai se fuder e me empresta sua netflix',
       'eu assisti 5 episódios de uma série na netflix e percebi que tava no 4g vsf',
       'estou assistindo as telefonistas, stranger things e how to get away with murder, além de ter vis a vis no backlog. daí vem a netflix e me lança nova temporada de elite! não to preparada!',
       '50 tons mais escuros vai ta na netflix nen adoro😅',
       'to tão por fora de tudo que eu vi hoje que saiu greys na netflix',
       'malta digam ai séries para ver na netflix',
       '@jcarloscr todas as séries originais amazon que eu assisti tinham uma qualidade superior na parte técnica. e gosto bastante dos roteiros mais diferenciados. é bem menos óbvio que a netflix',
       'e minha netflix q aparece que lancou a 15 temporada de greys e a 2 de elite e nada delas la',
       '@felipeneto @vislompo @thiagopinheiro2 @cardoso @abbondanzaisa @cris

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [229]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;\n#@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [230]:

data_clean=[]
for i in data_indexado.index:
    c= cleanup(i.lower())
    data_clean.append(c)
print(data_clean)


['milhões de séries disponíveis na netflix,mas minha preferência é sempre  glee   ', ' buriedslive vai se fuder e me empresta sua netflix', 'eu assisti 5 episódios de uma série na netflix e percebi que tava no 4g vsf', 'estou assistindo as telefonistas, stranger things e how to get away with murder, além de ter vis a vis no backlog  daí vem a netflix e me lança nova temporada de elite  não to preparada ', '50 tons mais escuros vai ta na netflix nen adoro😅', 'to tão por fora de tudo que eu vi hoje que saiu greys na netflix', 'malta digam ai séries para ver na netflix', ' jcarloscr todas as séries originais amazon que eu assisti tinham uma qualidade superior na parte técnica  e gosto bastante dos roteiros mais diferenciados  é bem menos óbvio que a netflix', 'e minha netflix q aparece que lancou a 15 temporada de greys e a 2 de elite e nada delas la', ' felipeneto  vislompo  thiagopinheiro2  cardoso  abbondanzaisa  cristiano_m_g  youpix a tv a cabo está sobrevivendo com ajuda de aparelho

In [231]:
data_rel=data_indexado.set_index('Relevância')
df_wie={'Frases':data_clean,
        'Relevância':data_rel.index
       }


In [232]:
df_clean = pd.DataFrame(df_wie)
df_clean_indexado=df_clean.set_index('Frases')
df_clean_indexado

,Relevância
Frases,
"milhões de séries disponíveis na netflix,mas minha preferência é sempre glee",2
buriedslive vai se fuder e me empresta sua netflix,0
eu assisti 5 episódios de uma série na netflix e percebi que tava no 4g vsf,0
"estou assistindo as telefonistas, stranger things e how to get away with murder, além de ter vis a vis no backlog daí vem a netflix e me lança nova temporada de elite não to preparada",1
50 tons mais escuros vai ta na netflix nen adoro😅,1
to tão por fora de tudo que eu vi hoje que saiu greys na netflix,2
malta digam ai séries para ver na netflix,0
jcarloscr todas as séries originais amazon que eu assisti tinham uma qualidade superior na parte técnica e gosto bastante dos roteiros mais diferenciados é bem menos óbvio que a netflix,0
e minha netflix q aparece que lancou a 15 temporada de greys e a 2 de elite e nada delas la,2


In [233]:
df_clean_indexado.Relevância.value_counts()
relevantes=df_clean_indexado[df_clean_indexado.Relevância==1]
irrelevantes=df_clean_indexado[df_clean_indexado.Relevância==0]
neutros=df_clean_indexado[df_clean_indexado.Relevância==2]


relevantes = ' '.join(relevantes.index)
irrelevantes = ' '.join(irrelevantes.index)
neutros = ' '.join(neutros.index)
print(relevantes[0:100]) 
print(irrelevantes[0:100]) 
print(neutros[0:100]) 


estou assistindo as telefonistas, stranger things e how to get away with murder, além de ter vis a v
 buriedslive vai se fuder e me empresta sua netflix eu assisti 5 episódios de uma série na netflix e
milhões de séries disponíveis na netflix,mas minha preferência é sempre  glee    to tão por fora de 


In [234]:
#https://pypi.org/project/emoji/
import functools
import operator
import re
import emoji

em = '😀👍😷'
em_split_emoji = emoji.get_emoji_regexp().split(em)
em_split_whitespace = [substr.split() for substr in em_split_emoji]
em_split = functools.reduce(operator.concat, em_split_whitespace)
#for oia in em_split:
print(em_split)

['😀', '👍', '😷']


In [235]:
import functools
import operator
import re
import emoji

relevantes_emojis=[]
relevantes = pd.Series(relevantes)
#print(relevantes)
for em in relevantes:
    em_split_emoji = emoji.get_emoji_regexp().split(em)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    for separated in em_split:
        relevantes_emojis.append(separated)
    #if len(em_split)==1:
        #relevantes_emojis.append(em_split)
    #else:
        #for im in em_split:
            #relevantes_emojis.append(im[0])
print(relevantes_emojis)
#relevantes_emojis_list=[]
#for em in relevantes_emojis:
#    if len(em)>1:
#        for im in em_split:
#            relevantes_emojis_list.append(im)
#    else:
#        relevantes_emojis_list.append(em)
#print(relevantes_emojis_list)
relevantes=pd.Series(relevantes_emojis)
relevantes.value_counts()
#relevantes_emojis_list[]

['estou', 'assistindo', 'as', 'telefonistas,', 'stranger', 'things', 'e', 'how', 'to', 'get', 'away', 'with', 'murder,', 'além', 'de', 'ter', 'vis', 'a', 'vis', 'no', 'backlog', 'daí', 'vem', 'a', 'netflix', 'e', 'me', 'lança', 'nova', 'temporada', 'de', 'elite', 'não', 'to', 'preparada', '50', 'tons', 'mais', 'escuros', 'vai', 'ta', 'na', 'netflix', 'nen', 'adoro', '😅', '2', 'vzs', 'foram', 'assisitindo', 'finding', 'dory', '(que', 'chegou', 'na', 'netflix)', 'e', '1', 'ouvindo', 'orange', 'juice', '🤧', '🤧', '🤧', 'colocaram', 'meu', 'filme', 'fav', 'na', 'netflix', 'aaaaaaa', 'é', 'hoje', 'que', 'eu', 'vou', 'assistir', 'o', 'mesmo', 'filme', '2763', 'vezes', 'tive', 'que', 'assinar', 'a', 'telecine', 'só', 'pra', 'assistir', 'amélie', 'poulain', 'pq', 'a', 'netflix', 'simplesmente', 'tirou', 'essa', 'obra', 'de', 'arte', 'do', 'catálogo', '😩', 'https', '//t', 'co/c3ouyeffbl', 'esse', 'mês', 'saí', 'a', 'quinta', 'temporada', 'de', 'b99', 'e', 'a', 'última', 'de', 'arthdral', 'chronic

netflix            172
de                  94
a                   92
e                   84
eu                  58
que                 54
na                  52
pra                 50
o                   41
só                  36
da                  29
é                   28
com                 27
não                 27
no                  26
uma                 25
assistir            25
um                  24
https               21
//t                 21
me                  18
do                  17
netflix,            17
vou                 17
em                  16
mas                 16
queria              16
q                   16
elite               16
esse                15
                  ... 
graças               1
wi                   1
prime                1
coloquei             1
romântico            1
festa                1
puta                 1
spotify,             1
sendonuma            1
usuário              1
anatomy,             1
meeeuuu              1
bebop      

In [236]:
relevantes_relativos=relevantes.value_counts(True)
relevantes_relativos

netflix            0.056747
de                 0.031013
a                  0.030353
e                  0.027714
eu                 0.019136
que                0.017816
na                 0.017156
pra                0.016496
o                  0.013527
só                 0.011877
da                 0.009568
é                  0.009238
com                0.008908
não                0.008908
no                 0.008578
uma                0.008248
assistir           0.008248
um                 0.007918
https              0.006928
//t                0.006928
me                 0.005939
do                 0.005609
netflix,           0.005609
vou                0.005609
em                 0.005279
mas                0.005279
queria             0.005279
q                  0.005279
elite              0.005279
esse               0.004949
                     ...   
graças             0.000330
wi                 0.000330
prime              0.000330
coloquei           0.000330
romântico          0

In [237]:
irrelevantes = pd.Series(irrelevantes)
irrelevantes_emojis=[]
for em in irrelevantes:
    em_split_emoji = emoji.get_emoji_regexp().split(em)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    for separated in em_split:
        irrelevantes_emojis.append(separated)
irrelevantes = pd.Series(irrelevantes_emojis)
irrelevantes_relativos=irrelevantes.value_counts(True)
irrelevantes_relativos

netflix          0.041176
a                0.029657
que              0.022059
de               0.022059
e                0.021569
na               0.017647
o                0.017402
não              0.013480
eu               0.013480
da               0.009069
tem              0.008578
https            0.008088
//t              0.008088
é                0.007843
no               0.007843
do               0.007598
uma              0.007598
com              0.007598
me               0.007353
ver              0.006863
em               0.006863
um               0.006863
vai              0.006373
pra              0.006373
mas              0.006127
já               0.005882
filme            0.005392
q                0.005392
minha            0.004902
se               0.004657
                   ...   
praia            0.000245
co/zpybx5qfl3    0.000245
floribella       0.000245
moral            0.000245
poupe            0.000245
apelido          0.000245
mcrivella        0.000245
co/v1ziivtq7

In [238]:
neutros = pd.Series(neutros)
neutros_emojis=[]
for em in neutros:
    em_split_emoji = emoji.get_emoji_regexp().split(em)
    em_split_whitespace = [substr.split() for substr in em_split_emoji]
    em_split = functools.reduce(operator.concat, em_split_whitespace)
    for separated in em_split:
        neutros_emojis.append(separated)
neutros = pd.Series(neutros_emojis)
neutros_relativos=neutros.value_counts(True)
neutros_relativos

netflix          0.052921
a                0.032951
eu               0.026960
de               0.023465
que              0.021468
e                0.016975
da               0.016475
na               0.016475
o                0.012481
só               0.011982
pra              0.010984
https            0.009985
//t              0.009985
temporada        0.008487
assistir         0.006990
uma              0.006990
com              0.006990
ver              0.006990
um               0.006990
não              0.006990
série            0.006490
minha            0.006490
to               0.006490
é                0.005991
do               0.005492
q                0.005492
me               0.005492
dia              0.004993
nao              0.004993
vou              0.004993
                   ...   
mean             0.000499
orgulho          0.000499
🍿                0.000499
omar             0.000499
glow             0.000499
lindo            0.000499
graças           0.000499
co/8l848jqiv

In [239]:
frases_treinamento=pd.read_excel(base,'Teste')
frase_indexado=frases_treinamento.set_index('Teste')
a=frase_indexado.index
frase_clean=[]
for i in a:
    c= cleanup(str(i).lower())
    frase_clean.append(c)
print(frase_clean)

[' petitluannes 15 que lançou agora no netflix kk', ' mananabranco hahahahah * melhor coisa do sábado cedinho , falar com a ma +fazer café +netflix', 'eu  gente o ander é obviamente o ativo, nunca que o omar come ele netflix  ander senta no omar eu  só acredito vendo eu  *vejo ainda eu  não acredito', 'agr posso assistir netflix em paz', 'esse tempinho em pleno sabado de feriado, só queria ficar deitada vendo netflix ', ' m_vasconcellus aí não dá pra ficar em casa dormindo e vendo só netflix', 'eu quando vou assistir netflix   99,9% procurando alguma coisa pra assistir 00,1% assistindo e depois eu termino nem assistindo e durmo', 'lost vai sair do catálogo da netflix mês que vem por causa do disney+ e mais uma vez os gays perdendo', 'estou muito feliz porque a netflix voltou com garotas malvadas, vou assistir mais 1899 vezes', 'caraca a pegação da nádia e do guzman foi muito além de um beijo, netflix sem limite', 'alguém conhece um site seguro pra ver série  quero convencer 2 amigos a 

In [240]:
class_relev={}
for frase1 in data_clean:  
    frase = frase1.split()
    log_probr = 0
    for p in frase:
        if p in relevantes_relativos:
            log_probr += np.log10(relevantes_relativos[p])
    log_probi = 0
    for p in frase:
        if p in irrelevantes_relativos:
            log_probi += np.log10(irrelevantes_relativos[p])
    log_probn = 0
    for p in frase:
        if p in neutros_relativos:
            log_probn += np.log10(neutros_relativos[p])
    log_probr*=-1
    log_probi*=-1
    log_probn*=-1
    if log_probr>log_probi and log_probr>log_probn:       
        class_relev[frase1]=1
    elif log_probi>log_probr and log_probi>log_probn:
        class_relev[frase1]=0
    else:
        class_relev[frase1]=2
print(class_relev)

{'milhões de séries disponíveis na netflix,mas minha preferência é sempre  glee   ': 2, ' buriedslive vai se fuder e me empresta sua netflix': 0, 'eu assisti 5 episódios de uma série na netflix e percebi que tava no 4g vsf': 0, 'estou assistindo as telefonistas, stranger things e how to get away with murder, além de ter vis a vis no backlog  daí vem a netflix e me lança nova temporada de elite  não to preparada ': 1, '50 tons mais escuros vai ta na netflix nen adoro😅': 1, 'to tão por fora de tudo que eu vi hoje que saiu greys na netflix': 1, 'malta digam ai séries para ver na netflix': 0, ' jcarloscr todas as séries originais amazon que eu assisti tinham uma qualidade superior na parte técnica  e gosto bastante dos roteiros mais diferenciados  é bem menos óbvio que a netflix': 0, 'e minha netflix q aparece que lancou a 15 temporada de greys e a 2 de elite e nada delas la': 2, ' felipeneto  vislompo  thiagopinheiro2  cardoso  abbondanzaisa  cristiano_m_g  youpix a tv a cabo está sobrevi

In [241]:
def classificar(frase_clean):
    class_relev1={}
    for frase1 in frase_clean:  
        frase = frase1.split()
        log_probr = 0
        for p in frase:
            if p in relevantes_relativos:
                log_probr += np.log10(relevantes_relativos[p])
        log_probi = 0
        for p in frase:
            if p in irrelevantes_relativos:
                log_probi += np.log10(irrelevantes_relativos[p])
        log_probn = 0
        for p in frase:
            if p in neutros_relativos:
                log_probn += np.log10(neutros_relativos[p])
        log_probr*=-1
        log_probi*=-1
        log_probn*=-1
        if log_probr>log_probi and log_probr>log_probn:       
            class_relev1[frase1]=1
        elif log_probi>log_probr and log_probi>log_probn:
            class_relev1[frase1]=0
        else:
            class_relev1[frase1]=2
    return class_relev1
class_relev1=classificar(frase_clean)

In [242]:
class_relevt= pd.DataFrame.from_dict(class_relev1,orient="index")
class_relevt

,0
petitluannes 15 que lançou agora no netflix kk,1
"mananabranco hahahahah * melhor coisa do sábado cedinho , falar com a ma +fazer café +netflix",1
"eu gente o ander é obviamente o ativo, nunca que o omar come ele netflix ander senta no omar eu só acredito vendo eu *vejo ainda eu não acredito",0
agr posso assistir netflix em paz,0
"esse tempinho em pleno sabado de feriado, só queria ficar deitada vendo netflix",1
m_vasconcellus aí não dá pra ficar em casa dormindo e vendo só netflix,0
"eu quando vou assistir netflix 99,9% procurando alguma coisa pra assistir 00,1% assistindo e depois eu termino nem assistindo e durmo",0
lost vai sair do catálogo da netflix mês que vem por causa do disney+ e mais uma vez os gays perdendo,0
"estou muito feliz porque a netflix voltou com garotas malvadas, vou assistir mais 1899 vezes",0
"caraca a pegação da nádia e do guzman foi muito além de um beijo, netflix sem limite",2


In [243]:
#sorte=class_relevt.merge(pd.DataFrame(data = frase_clean * len(frase_clean), columns = frase_clean.index), left_index=True, right_index=True)
wo=frase_indexado['Relevância'].tolist()
dicto={}
for w in range(0,len(frase_clean)):
    dicto[frase_clean[w]]=wo[w]
df_wo=pd.DataFrame.from_dict(dicto,orient="index")
df_wo


,0
petitluannes 15 que lançou agora no netflix kk,2
"mananabranco hahahahah * melhor coisa do sábado cedinho , falar com a ma +fazer café +netflix",1
"eu gente o ander é obviamente o ativo, nunca que o omar come ele netflix ander senta no omar eu só acredito vendo eu *vejo ainda eu não acredito",0
agr posso assistir netflix em paz,1
"esse tempinho em pleno sabado de feriado, só queria ficar deitada vendo netflix",1
m_vasconcellus aí não dá pra ficar em casa dormindo e vendo só netflix,1
"eu quando vou assistir netflix 99,9% procurando alguma coisa pra assistir 00,1% assistindo e depois eu termino nem assistindo e durmo",0
lost vai sair do catálogo da netflix mês que vem por causa do disney+ e mais uma vez os gays perdendo,0
"estou muito feliz porque a netflix voltou com garotas malvadas, vou assistir mais 1899 vezes",1
"caraca a pegação da nádia e do guzman foi muito além de um beijo, netflix sem limite",1


In [244]:
teste=df_wo.join(class_relevt, how='right', lsuffix='_determinados_a_mão',rsuffix='_classificados_pelo_computador')
teste

,0_determinados_a_mão,0_classificados_pelo_computador
petitluannes 15 que lançou agora no netflix kk,2,1
"mananabranco hahahahah * melhor coisa do sábado cedinho , falar com a ma +fazer café +netflix",1,1
"eu gente o ander é obviamente o ativo, nunca que o omar come ele netflix ander senta no omar eu só acredito vendo eu *vejo ainda eu não acredito",0,0
agr posso assistir netflix em paz,1,0
"esse tempinho em pleno sabado de feriado, só queria ficar deitada vendo netflix",1,1
m_vasconcellus aí não dá pra ficar em casa dormindo e vendo só netflix,1,0
"eu quando vou assistir netflix 99,9% procurando alguma coisa pra assistir 00,1% assistindo e depois eu termino nem assistindo e durmo",0,0
lost vai sair do catálogo da netflix mês que vem por causa do disney+ e mais uma vez os gays perdendo,0,0
"estou muito feliz porque a netflix voltou com garotas malvadas, vou assistir mais 1899 vezes",1,0
"caraca a pegação da nádia e do guzman foi muito além de um beijo, netflix sem limite",1,2


In [245]:
lista_certos=teste['0_determinados_a_mão'].tolist()
lista_testes=teste['0_classificados_pelo_computador'].tolist()
vitoria=0
verdadeiro_positivo=0
verdadeiro_negativo=0
falso_positivo=0
falso_negativo=0
verdadeiro_neutro=0
falso_neutro=0
for j in range(0,len(teste.index)):
    if lista_certos[j]==lista_testes[j]:
        vitoria+=1
    if lista_certos[j]==1 and lista_testes[j]==1:
        verdadeiro_positivo+=1
    elif lista_certos[j]==1 and lista_testes[j]!=1:
        verdadeiro_negativo+=1
    elif lista_certos[j]==0 and lista_testes[j]==0:
        falso_positivo+=1
    elif lista_certos[j]==0 and lista_testes[j]!=0:
        falso_negativo+=1
    elif lista_certos[j]==2 and lista_testes[j]==2:
        verdadeiro_neutro+=1
    elif lista_certos[j]==2 and lista_testes[j]!=2:
        falso_neutro+=1
print((vitoria/len(lista_certos))*100)
print('Logo, nossos testes tem 41% de acurácia de acordo com nossa classificação')

41.0
Logo, nossos testes tem 41% de acurácia de acordo com nossa classificação


In [246]:
import plotly.graph_objects as go

labels = ['Verdadeiro positivo','Verdadeiro negativo','Falso positivo','Falso negativo','Verdadeiro neutro','Falso neutro']
values = [verdadeiro_positivo, verdadeiro_negativo, falso_positivo, falso_negativo, verdadeiro_neutro, falso_neutro]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()

In [247]:
lista_neutros_testados=0
lista_relevantes_testados=0
lista_irrelevantes_testados=0
for j in lista_testes:
    if j==0:
        lista_irrelevantes_testados+=1
    elif j==1:
        lista_relevantes_testados+=1
    else:
        lista_neutros_testados+=1

In [248]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
def plotar(lista_relevantes_testados,lista_irrelevantes_testados,lista_neutros_testados):
    fig = make_subplots(rows=1, cols=2, specs=[[{"type": "bar"},
                                            {"type": "surface"}]])
    fig.add_trace(go.Bar(y=[lista_irrelevantes_testados],name="Irrelevantes"), row=1, col=1)
    fig.add_trace(go.Bar(y=[lista_relevantes_testados],name="Relevantes"), row=1, col=1)
    fig.add_trace(go.Bar(y=[lista_neutros_testados],name="Neutros"), row=1, col=1)
    fig.update_layout(showlegend=True)
    fig.show()
plotar(lista_relevantes_testados,lista_irrelevantes_testados,lista_neutros_testados)

In [249]:
import time

help(time.sleep)

Help on built-in function sleep in module time:

sleep(...)
    sleep(seconds)
    
    Delay execution for a given number of seconds.  The argument may be
    a floating point number for subsecond precision.



___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**